# Inference Batch Flow

This notebook shows how to create a Globus Flow to 1) transfer an input file to an HPC system and 2) run an inference on that HPC system. It also shows how to run the flow, check its status, and collect the result from the compute step.

In [ ]:
import os
import re
from dotenv import load_dotenv
from globus_sdk import FlowsClient, ClientApp, UserApp, SpecificFlowClient

### Create Authenticated Transfer Client

In [ ]:
# Load environment variables
load_dotenv(override=True)

# Extract the credentials of the Globus application that can trigger remote computations
CLIENT_ID = os.getenv("POLARIS_ENDPOINT_ID")
CLIENT_SECRET = os.getenv("POLARIS_ENDPOINT_SECRET")

# Extract who (Globus identities) can administer the flow
# FLOW_ADMINISTRATORS='
# urn:globus:auth:identity:<uuid-of-identity-1>
# urn:globus:auth:identity:<uuid-of-identity-2>
#'
FLOW_ADMINISTRATORS = re.split(r"[\s;]+", os.getenv("FLOW_ADMINISTRATORS").strip())

In [ ]:
# Create instance of ClientApp
client_app = ClientApp(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Create authenticated Flows client
flows_client = FlowsClient(app=client_app, app_name="Inference Batch Flow")

### Flow Definition

In [ ]:
# Inference flow definition
flow_definition = {
    "Comment": "Inference Flow Definition",
    "StartAt": "TransferFile",
    "States": {
        "TransferFile": {
            "Comment": "Transfer file from API's local storage to HPC facility",
            "Type": "Action",
            "ActionUrl": "https://transfer.actions.globus.org/transfer",
            "Parameters": {
                "source_endpoint.$": "$.input.source.id",
                "destination_endpoint.$": "$.input.destination.id",
                "DATA": [
                    {
                        "source_path.$": "$.input.source.path",
                        "destination_path.$": "$.input.destination.path",
                        "recursive": False,
                    }
                ]
            },
            "ResultPath": "$.TransferFile_output",
            "WaitTime": 18000,
            "Next": "ComputeInference"
        },
        "ComputeInference": {
            "Comment": "Run batch inference using transfered file",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/",
            "Parameters": {
                "endpoint.$": "$.input.compute_inference.endpoint",
                "function.$": "$.input.compute_inference.function",
                "args.$": "$.input.compute_inference.arguments"
            },
            "ResultPath": "$.ComputeInference_output",
            "WaitTime": 172800,
            "Next": "ShareResults"
        },
        "ShareResults": {
            "Comment": "Share output folder with user through Guest collection ACL rule",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/",
            "Parameters": {
                "endpoint.$": "$.input.share_results.endpoint",
                "function.$": "$.input.share_results.function",
                "args": [
                    {
                        "user_id.$": "$.input.share_results.user_id",
                        "username.$": "$.input.share_results.username",
                        "result.$": "$.ComputeInference_output.details.result[0]",
                    }
                ]
            },
            "ResultPath": "$.ShareResults_output",
            "WaitTime": 3600,
            "End": True
        }
    }
}

### Flow Input Schema Definition

In [ ]:
# Input schema definition
input_schema = {
    "required": [
        "input"
    ],
    "properties": {
        "input": {
            "type": "object",
            "required": [
                "source",
                "destination",
                "compute_inference",
                "share_results"
            ],
            "properties": {
                "source": {
                    "type": "object",
                    "title": "Source collection and path",
                    "description": "The source collection and path (path MUST end with a slash)",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "destination": {
                    "type": "object",
                    "title": "Destination collection and path",
                    "description": "The destination collection and path (path MUST end with a slash)",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "compute_inference": {
                    "type": "object",
                    "title": "Compute inference",
                    "description": "Compute inference endpoint and function",
                    "required": [
                        "endpoint",
                        "function",
                        "arguments"
                    ],
                    "properties": {
                        "endpoint": {
                            "type": "string",
                            "format": "uuid",
                            "title": "Compute endpoint ID",
                            "description": "The UUID of the Globus Compute endpoint"
                        },
                        "function": {
                            "type": "string",
                            "format": "uuid",
                            "title": "Compute function ID",
                            "description": "The UUID of the Globus Compute function to invoke"
                        },
                        "arguments": {
                            "type": "array",
                            "title": "Function arguments",
                            "description": "Inference function input arguments [{the_input_dictionary}]",
                            "items": {
                                "type": "object",
                                "required": [
                                    "model_params",
                                    "batch_id",
                                    "username"
                                ],
                                "properties": {
                                    "model_params": {
                                        "type": "object",
                                        "required": [
                                            "input_file",
                                            "model"
                                        ],
                                        "properties": {
                                            "input_file": {
                                                "type": "string",
                                                "title": "Input file",
                                                "description": "Full path of the input file on the HPC system"
                                            },
                                            "model": {
                                                "type": "string",
                                                "title": "Model",
                                                "description": "Selected large language model for the batch"
                                            },
                                            "output_folder_path": {
                                                "type": "string",
                                                "title": "Output folder path",
                                                "description": "Full path where the results will be written"
                                            }
                                        },
                                        "additionalProperties": False
                                    },
                                    "batch_id": {
                                        "type": "string",
                                        "title": "Inference batch ID",
                                        "description": "The UUID of the batch generated by the Inference API"
                                    },
                                    "username": {
                                        "type": "string",
                                        "title": "Username",
                                        "description": "Username (email address) of the user who submitted the batch"
                                    }
                                },
                                "additionalProperties": False
                            }
                        }
                    },
                    "additionalProperties": False
                },
                "share_results": {
                    "type": "object",
                    "title": "Share results",
                    "description": "Share output folder with user through Guest collection ACL rule",
                    "required": [
                        "endpoint",
                        "function",
                        "user_id",
                        "username",
                        #"results_file"
                    ],
                    "properties": {
                        "endpoint": {
                            "type": "string",
                            "format": "uuid",
                            "title": "Compute endpoint ID",
                            "description": "The UUID of the Globus Compute endpoint"
                        },
                        "function": {
                            "type": "string",
                            "format": "uuid",
                            "title": "Compute function ID",
                            "description": "The UUID of the Globus Compute function to invoke"
                        },
                        "user_id": {
                            "type": "string",
                            "title": "User Globus ID",
                            "description": "The Globus UUID of the user identity who submitted the batch"
                        },
                        "username": {
                            "type": "string",
                            "title": "Username",
                            "description": "Username (email address) of the user who submitted the batch"
                        }
                    },
                    "additionalProperties": False
                }
            },
            "additionalProperties": False
        }
    },
    "additionalProperties": False
}

### Register Flow

In [ ]:
# Create flow and assign permissions
flow = flows_client.create_flow(
    title="Inference Batch Flow", 
    definition=flow_definition, 
    input_schema=input_schema,
    flow_administrators=FLOW_ADMINISTRATORS
)

# Collect the flow UUID
flow_id = flow["id"]
print(f"Flow ID: {flow_id}")
print(f"https://app.globus.org/flows/{flow_id}")

### Test Flow

In [ ]:
# Get a Flows client for the targetted flow
specific_flow_client = SpecificFlowClient(flow_id=flow_id, app=client_app)

In [ ]:
# Input arguments and constants
input_file = "test.jsonl"
username = "user@this.is.a.test.com"
user_id = "globus-user-uuid"
eagle_base_path = "/eagle/projects/PortalDevelopment/inference_upload_test"
batch_id = "inference-batch-uuid"

# Define flow input 
flow_input = {
    "input": {
        "source": {
            "id": "transfer-endpoint-uuid-of-the-VM", # Not implemented yet
            "path": f"/{input_file}"
            
        },
        "destination": {
            "id": "5d64d93a-1293-4dae-a8c6-39d51daf2dd3", # Eagle - InferenceUpload
            "path": f"/uploaded_files/{input_file}"
        },
        "compute_inference": {
            "endpoint": "177b2ebc-e126-47d5-b753-6070cdbcae81", # sophia-vllm-batch-endpoint
            "function": "8c731d7a-b31f-4ebd-a824-858b6b4b4649", # vllm batch function
            "arguments": [
                {
                    "model_params": {
                        "input_file": f"{eagle_base_path}/uploaded_files/{input_file}",
                        "model": "meta-llama/Llama-3.3-70B-Instruct",
                        "output_folder_path": f"{eagle_base_path}/batch_results/{username}/"
                    },
                    "batch_id": batch_id,
                    "username": username,
                }
            ]
        },
        "share_results": {
            "endpoint": "c604bc6e-a4f1-44ff-b25a-7fd042257a98", # sophia-share-results-endpoint
            "function": "eeba0058-b9a6-44ec-a529-883488794b0e", # share_batch_results function (Sophia)
            "user_id": user_id, # User who submitted the request
            "username": username
        }
    }
}

In [ ]:
# Define flow label for this specific run
run_label = "Testing"

# Run the flow and include who can manage the run
run = specific_flow_client.run_flow(
    body=flow_input,
    label=run_label,
    run_managers=FLOW_ADMINISTRATORS
)

# Collect the run UUID
run_id = run["run_id"]
print(f"Run ID: {run_id}")
print(f"Check status at: https://app.globus.org/runs/{run_id}/logs")

### Get Flow results

In [ ]:
# Get flow status from Flows Client
flow_result = flows_client.get_run(run_id)

In [ ]:
# Check flow status
flow_result.data["status"]

In [ ]:
# Collect inference compute result
if flow_result.data["status"] == "SUCCEEDED":
    print(flow_result["details"]["output"]["ComputeInference_output"]["details"]["result"][0])

In [ ]:
# Collect share collection result
if flow_result.data["status"] == "SUCCEEDED":
    print(flow_result["details"]["output"]["ShareResults_output"]["details"]["result"][0])